# Worksheet

[FORM](https://forms.gle/uV4XBp3pNBktVdHU8)

(Note: the rest of this worksheet is ungraded).

## Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

Step 1: Assign each point to the nearest centroid:
For centroid 0: [0, .5] are closer to 0 than 2.
For centroid 2: [1.5, 2, 6, 6.5, 7] are closer to 2 than 0.
Clusters:
Cluster 1: [0, .5]
Cluster 2: [1.5, 2, 6, 6.5, 7]

Step 2: Recompute Centroids
For Cluster 1: New centroid = mean of [0, .5] = (0 + .5) / 2 = 0.25.
For Cluster 2: New centroid = mean of [1.5, 2, 6, 6.5, 7] = (1.5 + 2 + 6 + 6.5 + 7) / 5 = 4.6.

Step 3: Re-Assign Points to the new Centroids
For centroid 0.25: [0, .5] are still closer to 0.25 than 4.6.
For centroid 4.6: [1.5, 2, 6, 6.5, 7] are closer to 4.6 than 0.25.
New clusters:
Cluster 1: [0, .5]
Cluster 2: [1.5, 2, 6, 6.5, 7]

Step 4: Recompute Centroids
Same centroids as before: 0.25 and 4.6.
Since the centroids didn’t change, the algorithm converges, and the final clusters are:
Cluster 1: [0, .5]
Cluster 2: [1.5, 2, 6, 6.5, 7]

b) Describe in plain english what the cost function for k means is.

The cost function in K-Means is the sum of squared distances between each data point and its assigned centroid. The goal of the algorithm is to minimize this cost function by finding the optimal positions for the centroids, which minimizes the distance between the points in the clusters and their centroids.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

K-Means can produce different results depending on:

Initial centroids: The algorithm is sensitive to the starting positions of centroids. Different initial positions can lead to different final clusters.

Local minima: K-Means optimizes the cost function but can get stuck in local minima. Different initializations might result in different outcomes.

Data structure: If clusters are overlapping or the data has irregular shapes, K-Means might split the data in various ways.

d) Do you think Lloyd's Algorithm always converges?

Yes, Lloyd’s Algorithm always converges. However, while the algorithm always converges (i.e., it will stop when the centroids no longer change), it may not always converge to the global minimum. It might converge to a local minimum depending on the initial centroid positions.

e) Follow along in class the implementation of Kmeans

In [ ]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))


    def lloyds(self):
        # Randomly initialize k centroids
        centers = self.data[np.random.choice(range(len(self.data)), self.k, replace=False)]
        
        while True:
            # Assign each data point to the nearest centroid
            new_assignment = []
            for point in self.data:
                distances = [np.linalg.norm(point - center) for center in centers]
                new_assignment.append(np.argmin(distances))
            
            self.assignment = new_assignment
            self.snap(centers)
            
            # Recompute the centroids
            new_centers = np.array([self.data[np.array(self.assignment) == i].mean(axis=0) for i in range(self.k)])
            
            # If the centroids haven't changed, break the loop
            if np.all(centers == new_centers):
                break
            centers = new_centers
        return
            

kmeans = KMeans(X, 4)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)